## Generate VTK file from Bempp's solution

In [1]:
import numpy as np
import meshio
from bempp_pbs import preprocess

Could not find Gmsh.Interactive plotting and shapes module not available.


1. Convert potential unit to [kcal/(mol e)]

In [2]:
eps0 = 8.8541878128e-12 # F/m
coulomb_to_qe = 6.24150975e+18  # Coulomb to elementary charge
m_to_angstrom = 1e10            # meter to Angstrom

to_volt = m_to_angstrom / eps0 / coulomb_to_qe
volt_to_kcalpermole = 23.061    # eV to kcal/mol, or V to kcal/(mol*e)

coef = to_volt * volt_to_kcalpermole  # convert the unit of potential in bempp to kcal/(mol*e)

2. Read in mesh files and generate grid

In [4]:
face = '../mesh/surf_gs1.0_noter_split.face'  # .face file
vert = '../mesh/surf_gs1.0_noter_split.vert'  # .vert file

In [5]:
grid = preprocess.generate_grid(face, vert)   # use bempp to generate grid

3. Read in solution file (.npy) from Bempp

In [6]:
# read solution file
result = np.load('../runs/6CO8/direct/solution.npy')
potential = result[:len(result)//2] * coef
# derivative = result[len(result)//2:]

4. Write cell data to a VTK file

In [7]:
# generate points, cells, cell_data to create vtk using meshio
points = grid.vertices.T                      
cells = [("triangle", grid.elements.T.astype("int32"))]
cell_data = dict()
cell_data['potential'] = [potential]
# cell_data['derivative'] = [derivative]

meshio.write_points_cells(
    'zika.vtk',
    points=points,
    cells=cells,
    cell_data=cell_data
)

----------------

### Appendix A. Compute mesh density 

In [ ]:
face_file = '../mesh/surf_gs1.0_noter_split.face'  # .face file
vert_file = '../mesh/surf_gs1.0_noter_split.vert'  # .vert file
face = open(face_file, 'r').read()
vert = open(vert_file, 'r').read()
faces = np.vstack(np.char.split(face.split('\n')[0:-1]))[:,:3].astype(int) - 1
verts = np.vstack(np.char.split(vert.split('\n')[0:-1]))[:,:3].astype(float)

L0 = verts[faces[:, 1]] - verts[faces[:, 0]]
L2 = verts[faces[:, 0]] - verts[faces[:, 2]]
normal = np.cross(L0, L2)
area = np.sqrt(np.sum(normal**2, axis=1)) / 2

mesh_density = faces.shape[0] / area.sum()